In [1]:
### imports

# external modules
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

2022-06-16 20:38:24.462231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = True

# Control for the notebook - turn off user-friendly mode to enable faster runtimes
userfriendly = True

In [18]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = '../data/'

# Select a list of good runs to train on in development training_mode
# Should be validated by eye
trainrunsls = [
                    {"297057":[[-1]]}, 
#                    {"297099":[[-1]]}, 
                    {"297101":[[-1]]},
                    {"297113":[[-1]]}, 
                    {"297114":[[-1]]}, 
                    {"297175":[[-1]]},
                    {"297177":[[-1]]},
                    {"297179":[[-1]]}, 
                    {"297215":[[-1]]},
#                    {"297218":[[-1]]},
                    {"297225":[[-1]]},
                    {"297296":[[-1]]}, 
                    {"297411":[[-1]]},
                    {"297426":[[-1]]},
                    {"297431":[[-1]]},
                    {"297434":[[-1]]},
                    {"297468":[[-1]]},
                    {"297483":[[-1]]},
                    {"297486":[[-1]]},
                    {"297503":[[-1]]},
                    {"297557":[[-1]]},
                    {"297598":[[-1]]},
                    {"297604":[[-1]]},
                    {"297620":[[-1]]}, 
                    {"297659":[[-1]]},
                    {"297670":[[-1]]},
                    {"297674":[[-1]]},
                    {"297678":[[-1]]}, 
                    {"297722":[[-1]]},
                    {"298997":[[-1]]},
                    {"299061":[[-1]]},
                    {"299065":[[-1]]}, 
                    {"299067":[[-1]]},
                    {"299096":[[-1]]},
                    {"299149":[[-1]]},
                    {"299178":[[-1]]}, 
                    {"299184":[[-1]]},
                    {"299185":[[-1]]},
                    {"299327":[[-1]]},
                    {"299329":[[-1]]}, 
                    {"299480":[[-1]]} 
]
              

# Select a list of good runs to test on in development training_mode
# Should be validated by eye
goodrunsls = {'2017':{
                    "297057":[[-1]], 
                    "297099":[[-1]], 
                    "297101":[[-1]],
                    "297113":[[-1]], 
                    "297114":[[-1]], 
                    "297175":[[-1]],
                    "297177":[[-1]],
                    "297179":[[-1]], 
                    "297215":[[-1]],
                    "297218":[[-1]],
                    "297225":[[-1]],
                    "297296":[[-1]], 
                    "297411":[[-1]],
                    "297426":[[-1]],
                    "297431":[[-1]],
                    "297434":[[-1]],
                    "297468":[[-1]],
                    "297483":[[-1]],
                    "297486":[[-1]],
                    "297503":[[-1]],
                    "297557":[[-1]],
                    "297598":[[-1]],
                    "297604":[[-1]],
                    "297620":[[-1]], 
                    "297659":[[-1]],
                    "297670":[[-1]],
                    "297674":[[-1]],
                    "297678":[[-1]], 
                    "297722":[[-1]],
                    "298997":[[-1]],
                    "299061":[[-1]],
                    "299065":[[-1]], 
                    "299067":[[-1]],
                    "299096":[[-1]],
                    "299149":[[-1]],
                    "299178":[[-1]], 
                    "299184":[[-1]],
                    "299185":[[-1]],
                    "299327":[[-1]],
                    "299329":[[-1]], 
                    "299480":[[-1]]
              }
              
}


badrunsls = {'2017':
                {
                    #"297048":[[-1]],
                    #"297282":[[-1]],
                    #"297283":[[-1]],
                    #"297284":[[-1]],
                    #"297287":[[-1]],
                    #"297288":[[-1]],
                    #"297289":[[-1]],
                    "299316":[[-1]],
                    "299317":[[-1]],
                    "299318":[[-1]],
                    "299324":[[-1]],
                    "299326":[[-1]],
                    #"301086":[[88,126]],
                    #"301086":[[89,89]],
                    #"303948":[[1710,1710]],
                    "297047":[[-1]], #close but, true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297211":[[-1]], #Reconstructs well
#                   "299325":[[-1]], #Reconstructs well
                    "297664":[[-1]], #true bad for all 8
                    "299317":[[-1]], #true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                    "297502":[[-1]]
                },
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                {
                #"317479":[[-1]],
                "317480":[[-1]],
                "317481":[[-1]],
                "317482":[[-1]],
                #"319847":[[1,35]]
            }}


# The year and eras being used
year = '2017'
eras = ['B']

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [
    ['NormalizedHitResiduals_TIB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2',
     'NormalizedHitResiduals_TIB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3' , 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'],
    ['chargeInner_PXLayer_1', 'chargeOuter_PXLayer_1', 'adc_PXLayer_1'],
    ['chargeInner_PXLayer_2', 'chargeOuter_PXLayer_2', 'adc_PXLayer_2'],
    ['chargeInner_PXLayer_3', 'chargeOuter_PXLayer_3', 'adc_PXLayer_3'],
    ['chargeInner_PXLayer_4', 'chargeOuter_PXLayer_4', 'adc_PXLayer_4'],
    ['charge_PXDisk_+1', 'adc_PXDisk_+1'],
    ['charge_PXDisk_-1', 'adc_PXDisk_-1'],
    ['charge_PXDisk_+2', 'adc_PXDisk_+2'],
    ['charge_PXDisk_-2', 'adc_PXDisk_-2'],
    ['charge_PXDisk_+3', 'adc_PXDisk_+3'],
    ['charge_PXDisk_-3', 'adc_PXDisk_-3'],
    ['NormalizedHitResiduals_TOB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'NormalizedHitResiduals_TOB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2',
     'NormalizedHitResiduals_TOB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3' , 'NormalizedHitResiduals_TOB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4']
]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [4]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Set whether to train globally or locally or in a development/testing mode
training_mode = 'development'

# Selects whether to create a new histstruct or use a saved one
readnew = True

# Select whether to save a new histstruct
save = False

In [5]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

# Evaluate models seperately, as an ensemble, both, or neither
individualEval = True
ensembleEval = False

In [6]:
def define_combined_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            ## Model parameters
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(256, activation="tanh")(conc_layer)
            encoder = Dense(128, activation='relu')(encoder)
            
            encoder = Dense(64, activation='relu')(encoder)
            
            decoder = Dense(128, activation="relu")(encoder)
            decoder = Dense(256, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [7]:
### Trains a combined autoencoder for every merge set
def train_combined_autoencoder(histslist, vallist, autoencoders):
    
    # Iterate through the training data to train corresponding autoencoders
    for i in range(len(histslist)):
        
        if userfriendly: print('\nNow training model {}/'.format(i + 1) + str(len(histslist)))
        
        # Set variables to temporary values for better transparency
        X_train = histslist[i]
        X_val = vallist[i]
        autoencoder = autoencoders[i]
        
        
        ## Model parameters
        nb_epoch = 200
        batch_size = 100
        
        #checkpoint_filepath = 'checkpoint'
        #model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #   filepath=checkpoint_filepath,
        #   save_weights_only=False,
        #   verbose=1,
        #   save_best_only=True,
        #   monitor='val_loss',
        #   mode='min')
        
        # Tell the model when to stop
        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=1e-7,
            patience=20,
            verbose=0,
            mode='auto',
            baseline=None,
            restore_best_weights=True,
        )
        lr =0.001
        opt = keras.optimizers.Adam(learning_rate=lr)
        
        autoencoder.compile(loss='mse',
                            optimizer=opt)
        
        ## Train autoencoder
        train = autoencoder.fit(x=[X_train[:,i] for i in range(X_train.shape[1])],
                                y=[X_train[:,i] for i in range(X_train.shape[1])],
                            epochs=nb_epoch,
                            batch_size=batch_size,
                            shuffle=True,
                            validation_data=([X_val[:,i] for i in range(X_val.shape[1])], [X_val[:,i] for i in range(X_val.shape[1])]),
                            verbose=0,
                                callbacks= [earlystop] 
                            )
        
        # Save classifier for evaluation
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier)        

In [8]:
### Evaluate the Models for WP definition
def evaluate_models_train(histstruct):
    
    for histgroup in histnames:
        histstruct.evaluate_classifier(histgroup)
    
    # get mse for training set
    if 'training' in histstruct.masks.keys(): masknames = ['dcson','highstat', 'training']
    else: masknames = ['dcson','highstat']
    mse_train = histstruct.get_scores_array( masknames=masknames )
    
    # get mse for good set
    if 'good' in histstruct.masks.keys():
        mse_good = []
        for histname in histstruct.histnames:
            mse_good.append(histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] ))
    else:
        mse_good = []
        for histname in histstruct.histnames:
            hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
            thismse = histstruct.classifiers[histname].evaluate( hists_good )
            mse_good.append( thismse )
            
    mse_good = np.array(mse_good)
    mse_good = np.transpose(mse_good)
    
    # get mse for bad sets
    mse_bad = []
    for i in range(nbadruns):
        mse_bad.append( histstruct.get_scores_array( masknames=['dcson','highstat','bad{}'.format(i)] ) )
        
    return [mse_train, mse_good, mse_bad]

In [9]:
### Plots and Distribution Analysis
def fit_mse_distribution(histstruct):
    dimslist = []
    fitfunclist = []
    
    
    nhisttypes = len(histstruct.histnames)
    for i in range(0,nhisttypes-1):
        for j in range(i+1,nhisttypes):
            dimslist.append((i, j))
    
    plt.close('all')
    (npoints,ndims) = mse_train.shape
    
    
    # settings for GaussianKdeFitter
    scott_bw = npoints**(-1./(ndims+4))
    bw_method = 20*scott_bw
    # settings for HyperRectangleFitter
    quantiles = ([0.00062,0.0006,0.00015,0.00015,
                 0.0003,0.0003,0.00053,0.00065])
    
    
    for dims in dimslist:
        thismse = mse_train[:,dims]
        if training_mode=='global': 
            fitfunc = SeminormalFitter.SeminormalFitter(thismse)
            #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
            #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
            #                                                    'up')
        else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
        #pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
        #                onlycontour=False, xlims=30, ylims=30, 
        #                onlypositive=True, transparency=0.5,
        #                xaxtitle=histstruct.histnames[dims[0]], 
        #                yaxtitle=histstruct.histnames[dims[1]],
        #                title='density fit of lumisection MSE')
        ##plt.close('all') # release plot memory
        fitfunclist.append(fitfunc)
     
        
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(mse_train,bw_method=bw_method)
    
    return fitfunc

In [10]:
### Prepare MSEs for Working Point Definition
def mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc):
    
    # Get the minimum log probability of histograms in good set
    logprob_good = np.log(fitfunc.pdf(mse_good_eval))
    #print(sorted(logprob_good))
    
    logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
    #for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
    logprob_bad = np.concatenate(tuple(logprob_bad_parts))
    
    return [logprob_good, logprob_bad]

In [11]:
def evaluate_autoencoder_individual(histstruct):
    
    msewps = []
    for histname in histstruct.histnames:
        
        # Get histograms from histstruct
        X_test_good = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'good']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
        
        X_test_bad = X_test_bad = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'bad']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
        
        # Get each model from the histstruct
        autoencoder = histstruct.get_classifier(histname)
        
        # Getting evaluation criteria
        prediction_test_good = autoencoder.reconstruct(X_test_good)
        mse_test_good = aeu.mseTopNRaw(X_test_good, prediction_test_good, n=10 )
        prediction_test_bad = autoencoder.reconstruct(X_test_bad)
        mse_test_bad = aeu.mseTopNRaw(X_test_bad, prediction_test_bad, n=10 )
        
        
        if createPlots:
            # Number of plots of each type to generate per model (so nplot * 2 * len(model))
            nplot = 3
            
            # Good examples
            print('Examples of good histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_good)),size=nplot,replace=False)
            for i in randint: 
                histlist = [X_test_good[int(i),:],prediction_test_good[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
            
            # Bad examples
            print('Examples of bad histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_bad)),size=nplot,replace=False)
            for i in randint:
                histlist = [X_test_bad[int(i),:],prediction_test_bad[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
        
        # Attaching the bad histograms as a new set of rows under the good histograms
        validation_data = np.vstack((X_test_good, X_test_bad))
        validation_preds = np.vstack((prediction_test_good, prediction_test_bad))
        # Creating labels to differentiate the data when we go to compare predictions
        #     with actual label
        labels = np.hstack((np.zeros(len(X_test_good)), np.ones(len(X_test_bad))))
        
        # Pick a working point to see 
        msewp = 0.5*(np.mean(mse_test_bad) - np.mean(mse_test_good))
        print("Selected working point: " + str(msewp))
        
        # Get data to pick a good working point for future evaluation
        scores = aeu.mseTop10Raw(validation_data, validation_preds)
        nsig = np.sum(labels)
        nback = np.sum(1-labels)
        
        # Get some metrics for the user
        tp = np.sum(np.where((labels==1) & (scores>msewp),1,0))/nsig
        fp = np.sum(np.where((labels==0) & (scores>msewp),1,0))/nback
        tn = 1-fp
        fn = 1-tp
        
        accuracy = (tp + tn) / (tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*precision*recall) / (precision + recall)
        
        if userfriendly:
            print(accuracy)
            print(precision)
            print(recall)
            print(f1)
        
    return msewps

In [15]:
def evaluate_autoencoders_combined(logprob_good, logprob_bad):
    labels_good = np.zeros(len(logprob_good)) # background: label = 0
    labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
    
    labels = np.concatenate(tuple([labels_good,labels_bad]))
    scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
    scores = aeu.clip_scores( scores )
    
    #pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
    #                   bcklabel='good', bckcolor='g', 
    #                   nbins=200, normalize=True,
    #                   xaxtitle='negative logarithmic probability',
    #                   yaxtitle='number of lumisections (normalized)')
    
    logprob_good[logprob_good > 500] = -500
    logprob_bad[logprob_bad < 0] = 0
    logprob_good[logprob_good < 0] = 0
    logprob_bad[logprob_bad > 500] = 500
    
    # Setting a threshold, below this working point defines anomalous data
    logprob_threshold = 0.5 * (np.mean(logprob_good) - np.mean(logprob_bad))
    # Or set manual

    # Get metrics for analysis
    (tp, fp, tn, fn) = get_confusion_matrix(scores,labels,-logprob_threshold)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_measure = (2 * precision * recall) / (precision + recall)
    print('Selected logprob threshold of ' + str(logprob_threshold))

    
    return [accuracy, recall]

In [16]:
def get_confusion_matrix(scores, labels, wp):
    nsig = np.sum(labels)
    nback = np.sum(1-labels)

    # get confusion matrix entries
    tp = np.sum(np.where((labels==1) & (scores>wp),1,0))/nsig
    fp = np.sum(np.where((labels==0) & (scores>wp),1,0))/nback
    tn = 1-fp
    fn = 1-tp
    cmat = np.array([[tp,fn],[fp,tn]])
    
    return(tp, fp, tn, fn)

In [19]:
bestRun = [{}, 0, 0]
for i, run in enumerate(trainrunsls):
    
    print('Run: {}'.format(run))
    
    # Select runs to be used in training from the user-defined list
    runsls_training = run
    # Select bad runs to test on in the user-defined list
    runsls_bad = badrunsls['2017']
    # Select good runs to test on in the user-defined list
    runsls_good = goodrunsls['2017']
        
    
    # Initializations
    dloader = DataLoader.DataLoader()
    histstruct = SubHistStruct.SubHistStruct()

    save_stdout = sys.stdout
    sys.stdout = open('trash', 'w')
    histstruct.reset_histlist(histnames)
    sys.stdout = save_stdout
    
    # Unpack histnames and add every histogram individually
    for histnamegroup in histnames:
        for histname in histnamegroup:
            
            # Bring the histograms into memory from storage for later use
            filename = '../data/DF'+year+'B_'+histname+'.csv'
            df = dloader.get_dataframe_from_file( filename )
            
            # In case of local training, we can remove most of the histograms
            if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
                runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
                df = dfu.select_runsls( df, runsls_total )
                
            # Store the data in the histstruct object managing this whole thing
            save_stdout = sys.stdout
            sys.stdout = open('trash', 'w')
            histstruct.add_dataframe( df, rebinningfactor = 3 )
            sys.stdout = save_stdout
            
    # Add masks
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat' )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=100 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
        
    # Distinguishing bad runs
    nbadruns = 0
    if runsls_bad is not None:
        histstruct.add_json_mask( 'bad', runsls_bad )
        
        # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
            
    (histslist, vallist, autoencoders, train_normhist) = define_combined_autoencoder(histstruct)
    
    print('\nTraining models...\n')
    save_stdout = sys.stdout
    sys.stdout = open('trash', 'w')
    train_combined_autoencoder(histslist, vallist, autoencoders)
    sys.stdout = save_stdout
    (mse_train, mse_good_eval, mse_bad_eval) = evaluate_models_train(histstruct)
    fitfunc = fit_mse_distribution(histstruct)
    (logprob_good, logprob_bad) = mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc)
    (accuracy, recall) = evaluate_autoencoders_combined(logprob_good, logprob_bad)
    
    print('Accuracy: ' + str(accuracy))
    print('Recall: ' + str(recall))
    
    if recall > bestRun[1]:
        bestRun[0] = run
        bestRun[1] = recall
        bestRun[2] = accuracy
        print('New Best Run: {}'.format(run))
    elif recall == bestRun[1]:
        if accuracy > bestRun[2]:
            bestRun[0] = run
            bestRun[1] = recall
            bestRun[2] = accuracy
            print('New Best Run: {}'.format(run))
    
    print('\n--------------------------------------\n')

print('Best Run: ' + bestRun)

Run: {'297057': [[-1]]}

Training models...



/tmp/ipykernel_71773/3406801837.py:5: RuntimeWarning: divide by zero encountered in log
  logprob_good = np.log(fitfunc.pdf(mse_good_eval))
/tmp/ipykernel_71773/3406801837.py:8: RuntimeWarning: divide by zero encountered in log
  logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]


NOTE: scores of +inf were reset to 332.8092645409154
Selected logprob threshold of 51.32703952568024
Accuracy: 0.6319083368403904
Recall: 0.9831932773109243
New Best Run: {'297057': [[-1]]}

--------------------------------------

Run: {'297101': [[-1]]}

Training models...



/tmp/ipykernel_71773/3406801837.py:5: RuntimeWarning: divide by zero encountered in log
  logprob_good = np.log(fitfunc.pdf(mse_good_eval))
/tmp/ipykernel_71773/3406801837.py:8: RuntimeWarning: divide by zero encountered in log
  logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]


NOTE: scores of +inf were reset to 325.8558775083542
Selected logprob threshold of 51.43129552378572
Accuracy: 0.6187263273645485
Recall: 0.9747899159663865

--------------------------------------

Run: {'297113': [[-1]]}

Training models...



...

/tmp/ipykernel_71773/3406801837.py:5: RuntimeWarning: divide by zero encountered in log
  logprob_good = np.log(fitfunc.pdf(mse_good_eval))
/tmp/ipykernel_71773/3406801837.py:8: RuntimeWarning: divide by zero encountered in log
  logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]


NOTE: scores of +inf were reset to 306.53050024603476
Selected logprob threshold of 1.8597102130534784
Accuracy: 0.5130462524266572
Recall: 0.9915966386554622

--------------------------------------

Run: {'297215': [[-1]]}

Training models...



/tmp/ipykernel_71773/3406801837.py:5: RuntimeWarning: divide by zero encountered in log
  logprob_good = np.log(fitfunc.pdf(mse_good_eval))
/tmp/ipykernel_71773/3406801837.py:8: RuntimeWarning: divide by zero encountered in log
  logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
/tmp/ipykernel_71773/2997766680.py:27: RuntimeWarning: invalid value encountered in double_scalars
  precision = tp / (tp + fp)


NOTE: scores of +inf were reset to -380.6419637566345
Selected logprob threshold of 1.3607534727319832
Accuracy: 0.5
Recall: 0.0

--------------------------------------

Run: {'297225': [[-1]]}

Training models...



LinAlgError: 24-th leading minor of the array is not positive definite